# Simulation of Stepper Control methods

In [1]:
import numpy as np
import matplotlib.pyplot as plt

### Input will be a frequency with a time period
stepFunc( frequency, time_to_stay): 
    - frequency: the next frequency (key) to which the gantry needs to move
    - time_to_stay: The time that the gantry must spend at the key
    
*** Maybe add volume control and key press duration to single function ***

In [ ]:
class stepperMotor(self, calibrate_f):
    def __init__(self, calibrate_f):
        self.calibration_f = calibrate_f
        self.previous_frequency = calibrate_f
        autohome(calibrate_f)
        
    def autohome(cal_freq):
        ## Homing function moves gantry until distance sensor reads out that the gantry 
        ## is at the end of the keyboard
    def stepFunc(frequency, time_to_stay):
        self.previous_frequency = self.calibration_f # calibration frequency will be the highest/lowest frequency where the
                                                     # gantry will begin from
        self.delta_f = previous_frequency - frequency
        if delta_f < 0 :
            print("DIR PIN SET TO LOW")
            stepMotor(delta_f)
        elif delta_f > 0:
            stepMotor(delta_f)
            print("DIR PIN SET TO HIGH")
        else:
            stepMotor(stay)
        self.previous_frequency = frequency
    
    def stepMotor(delta_freq):
        

In [ ]:
def dirEn(direction):
    global state
    if (direction == -1) & (state == 0):
        state = 1
        GPIO.output(DIR, GPIO.HIGH)
    elif (direction == -1) & (state ==1):
        state = 0
        GPIO.output(DIR, GPIO.LOW)
    return state
    